# Analisi infetti e deceduti

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from aidam.math_utils import moving_function_rev1
from aidam.aicharts import get_subplot_cells_sequence, confplot_ax, auto_grid_shape
import pickle as pkl
import numpy as np
import pandas as pd

## Lettura dati

In [2]:
data_file='stored_data.pkl'

infile=open(data_file,'rb')
trend=pkl.load(infile)
regioni=pkl.load(infile)
giorni=pkl.load(infile)
giorno_settimana=pkl.load(infile)
popolazione=pkl.load(infile)
infile.close()

day_counter=list(range(len(giorni)))

print('Giorni osservati: %d'%len(giorni))
print('Primo giorno: %s'%giorni[0])
print('Ultimo giorno: %s'%giorni[-1])

Giorni osservati: 206
Primo giorno: 2020-02-24T18:00:00
Ultimo giorno: 2020-09-16T17:00:00


### Inserimento di eventi notevoli riferiti a date

In [3]:
lockdown={'data':'2020-03-10',
          'id_giorno':15,
          'desc':'Lockdown'}

scuole_chiuse={'data':'2020-03-04',
               'id_giorno':9,
               'desc':'Scuole chiuse'}

fase_2={'data':'2020-05-04',
               'id_giorno':71,
               'desc':'Fase2'}


eventi=[scuole_chiuse,lockdown, fase_2]

Calcolo le posizioni nella grid dei subplot dove mettere le figure delle varie regioni e parametri delle figure

In [4]:
# per i subplot determina la griglia
grid_shape=(7,3)
grid_seq=get_subplot_cells_sequence(grid_shape)

# dimensioni figure (figsize) per figure singole e figure composte (subplot)
figsize_single=(9,6)
figsize_multi=(16,16)


## Grafici Nazionali

In [5]:
indicatori_selected=['totale_casi',
                     'nuovi_positivi',
                     'guariti_giornalieri',
                    'variazione_totale_positivi',
                    'totale_positivi',
                    'deceduti_giornalieri']

In [6]:
for indicatore in indicatori_selected:
    fig,ax=plt.subplots(1,figsize=figsize_single)
    # stampo il plot smooth con intervallo di confidenza
    confplot_ax(day_counter,
                   trend[indicatore]['Italia'],
                   window=(3,3),ax=ax,conf_fun='deviation', )
    ax.set_title(indicatore+' - Odierno: %.1f'%trend[indicatore]['Italia'][-1])
    ax.set_xlabel('Giorni dal %s al %s'%(giorni[0],giorni[-1]))
    ax.set_ylabel(indicatore)
    # stampo anche a punti il dato raw
    ax.plot(day_counter,trend[indicatore]['Italia'],'b.')
    ax.legend(['Trend','Low','Up','Raw'])
    ax.grid()
    ax.set_xticks(list(range(0,len(giorni),10)))
    ax.set_xticklabels(giorni[list(range(0,len(giorni),10))],rotation=90)



In [7]:
indicatori_selected=['totale_positivi',
                     'deceduti_giornalieri',
                     'terapia_intensiva']

In [8]:
fig,ax=plt.subplots(1,3,figsize=(20,5))
for i,indicatore in enumerate(indicatori_selected):  
    ax[i].plot(day_counter,
             trend[indicatore]['Italia'],label=indicatore)
    ax[i].set_xticks(list(range(0,len(giorni),10)))
    ax[i].set_xticklabels(giorni[list(range(0,len(giorni),10))],rotation=90)
    ax[i].axvline(x=lockdown['id_giorno'],c='r', label='Lockdown',linewidth=4,ls='--')
    ax[i].axvline(x=scuole_chiuse['id_giorno'],c='c', label='Scuole chiuse',linewidth=4,ls='--')
    ax[i].axvline(x=fase_2['id_giorno'],c='g', label='Fase 2',linewidth=4,ls='--')
    ax[i].grid()
    ax[i].legend()

### Grafici per regione

In [9]:
indicatori_selected=['nuovi_positivi',
                    'guariti_giornalieri',
                    'totale_positivi']


for indicatore in indicatori_selected:

    fig,ax=plt.subplots(grid_shape[0],grid_shape[1],sharey=False,figsize=figsize_multi)
    fig.suptitle(indicatore, y=1.02,fontsize=25)
    fig.tight_layout(pad=2.0)

    for i,regione in enumerate(regioni):
        ax[grid_seq[i]].plot(day_counter,trend[indicatore][regione],
                             'b-',linewidth=0.5,label='Reg')
        ax[grid_seq[i]].plot(day_counter,moving_function_rev1(trend[indicatore][regione],np.mean,3,3),
                             'b--',linewidth=2,label='Trend')
        ax[grid_seq[i]].set_title(regione+' Odierno: %.1f'%trend[indicatore][regione][-1])
        #ax[grid_seq[i]].legend()
        ax[grid_seq[i]].grid()
    fig.savefig('PIC_%s.JPG'%indicatore)

## Deceduti nel tempo
**Italia Giornalieri**

In [10]:
fig,ax=plt.subplots(1,figsize=figsize_single)
# stampo il plot smooth con intervallo di confidenza
confplot_ax(day_counter,
            trend['deceduti_giornalieri']['Italia'],
            window=(3,3),
            ax=ax,conf_fun='deviation', )
ax.set_title('Deceduti giornalieri in Italia')
ax.set_xlabel('Giorni dal %s al %s'%(giorni[0],giorni[-1]))
ax.set_ylabel('Decessi')
# stampo anche a punti il dato raw
ax.plot(trend['deceduti_giornalieri']['Italia'],'b.')
ax.legend(['Trend','Low','Up','Raw'])
ax.set_xticks(list(range(0,len(giorni),10)))
ax.set_xticklabels(giorni[list(range(0,len(giorni),10))],rotation=45)
ax.grid()

**Per regione giornalieri**

In [11]:
fig,ax=plt.subplots(grid_shape[0],grid_shape[1],sharey=False,figsize=figsize_multi)
fig.suptitle('deceduti_giornalieri', y=1.02,fontsize=25)
fig.tight_layout(pad=2.0)

for i,regione in enumerate(regioni):
    ax[grid_seq[i]].plot(day_counter,trend['deceduti_giornalieri'][regione],'b-',label='Raw')
    ax[grid_seq[i]].set_title(regione+' Odierno: %.1f'%trend['deceduti_giornalieri'][regione][-1])
    # plotto anche media
    ax[grid_seq[i]].plot(day_counter,
                         moving_function_rev1(trend['deceduti_giornalieri'][regione],np.mean,3,3),
                         'g--',
                         label='Trend')
    ax[grid_seq[i]].legend()

### Indicatori per regione negli ultimi giorni

Si specifica la lista degli indicatori e i giorni indietro cui guardare per la produzione di grafici a barre

In [12]:
indicatori_selected=['nuovi_positivi','totale_positivi','deceduti_giornalieri']

ultimi_giorni=14
for indicatore in indicatori_selected:
    # calcolo la media da mostrare
    valori=[np.mean(trend[indicatore][r][-ultimi_giorni:])/popolazione[regione] for r in regioni]
    plt.figure(figsize=(12,5))
    plt.title('Indicatore [%s] su popolazione. Media ultimi %d giorni'%(indicatore,ultimi_giorni))
    plt.bar(range(len(regioni)),valori)
    plt.xticks(range(len(regioni)),regioni,rotation=90)
    plt.grid()
    plt.show()

### Situazione regione singola

In [13]:
# Selezione regione e indicatori da plottare

regione='Toscana'

indicatori_selected=['nuovi_positivi',
                     'guariti_giornalieri',
                     'deceduti_giornalieri',
                     'totale_positivi','variazione_totale_positivi']

num_indicatori=len(indicatori_selected)

In [14]:
trend['variazione_totale_positivi'][regione]

array([   0,    2,    0,    0,    5,    3,    2,    0,    6,   19,   23,
         18,   34,   53,   41,   54,   54,   38,  103,  159,  149,   78,
        183,  267,  131,  291,  192,  239,  157,  218,  257,  197,  197,
        341,  275,  264,  176,  206,  357,  120,  145,  131,  116,  126,
        130,  146,  119,  170,  170,   95,   95,   65,  196,  -30, -113,
         26,   72,   54, -455,    4,  -38,   13,  -77,  -86,  -87, -233,
        -79, -211,   -8,  -37,  -49,  -89, -102, -372, -124, -144, -301,
        -74, -232, -278, -175, -220, -225, -141, -229, -250, -206, -240,
        -91,  -20,  -66,  -64, -114,  -62,  -80, -125,  -89,  -55,  -29,
        -71,  -38,  -90,  -26,  -72,  -35,  -29,  -93,  -89,  -18,  -11,
         -8,   -3,  -10,  -26,  -19,   -6,  -15,   -9,  -49,   -5,  -23,
         -7,   -6,  -10,    4,    5,    4,    2,   -6,    1,   -1,    1,
          6,    2,    7,    2,   -4,   -7,    0,    1,    4,  -24,    2,
          2,    3,   -4,   11,   -1,    2,    6,   

In [15]:
grid_shape=auto_grid_shape(num_indicatori,cols=2)
grid_seq=get_subplot_cells_sequence(grid_shape)

fig,ax=plt.subplots(grid_shape[0],grid_shape[1],figsize=figsize_multi)
fig.suptitle(regione, y=1.02,fontsize=25)
fig.tight_layout(pad=2.0)


for i,indicatore in enumerate(indicatori_selected):
    ax[grid_seq[i]].plot(day_counter,trend[indicatore][regione],
                         'b-',linewidth=0.5,label='Reg')
    ax[grid_seq[i]].plot(day_counter,moving_function_rev1(trend[indicatore][regione],np.mean,3,3),
                         'b--',linewidth=2,label='Trend')
    ax[grid_seq[i]].set_title(indicatore+' - Odierno: %.1f'%trend[indicatore][regione][-1])
    #ax[grid_seq[i]].legend()
    ax[grid_seq[i]].grid()

### Relazione fra deceduti e terapia intensiva

In [16]:
from sklearn.linear_model import LinearRegression

var_indip='terapia_intensiva'

lr=LinearRegression()
lr.fit(trend[var_indip]['Italia'].reshape(-1, 1),
         trend['deceduti_giornalieri']['Italia'])

newx=np.linspace(0,max(trend[var_indip]['Italia'])).reshape(-1,1)

lry=lr.predict(newx)

In [17]:
plt.figure(figsize=figsize_single)
plt.plot(trend[var_indip]['Italia'],
         trend['deceduti_giornalieri']['Italia'],
        'bo',label='Data',)

plt.plot(newx,
         lry,
        'g--',label='Fit',)

plt.plot(trend[var_indip]['Italia'][-1],
         trend['deceduti_giornalieri']['Italia'][-1],
        'rd',label=giorni[-1],)


plt.title('Deceduti vs %s'%var_indip)
plt.xlabel(var_indip)
plt.ylabel('Decessi')
plt.legend()
plt.grid()

## Deceduti vs Guariti

Trend della percentuale giornaliera di **deceduti** e **guariti** rispetto al numero di *attualmente* infetti (al giorno precedente)

In [18]:
fig,ax=plt.subplots(1,figsize=figsize_single)
# stampo il plot smooth con intervallo di confidenza
confplot_ax(day_counter,
            trend['perc_guariti_su_positivi']['Italia'],
            window=(10,10),
            ax=ax,conf_fun='deviation',color='g')
confplot_ax(day_counter,
            trend['perc_deceduti_su_positivi']['Italia'],
            window=(7,7),
            ax=ax,conf_fun='deviation',color='r')

ax.set_title('Percentuale guariti e deceduti rispetto a positivi')
ax.set_xlabel('Giorni dal %s al %s'%(giorni[0],giorni[-1]))
ax.set_ylabel('%')
# stampo anche a punti il dato raw
#ax.plot(trend['perc_guariti_su_positivi']['Italia'],'g.')
#ax.plot(trend['perc_deceduti_su_positivi']['Italia'],'r.')
ax.legend(['% Trend guariti su positivi','High','Low',
           '% Trend deceduti su positivi','High','Low',
           '% Raw guariti su positivi',
           '% Raw deceduti su positivi'])
ax.grid()

In [19]:
fig,ax=plt.subplots(1,figsize=figsize_single)
# stampo il plot smooth con intervallo di confidenza
confplot_ax(day_counter,
            trend['perc_guariti_su_positivi']['Italia']/trend['perc_deceduti_su_positivi']['Italia'],
            window=(3,3),
            ax=ax,conf_fun='deviation')
ax.set_title('Rapporto fra guariti e deceduti rispetto ai positivi giornalieri')
ax.set_xlabel('Giorni dal %s al %s'%(giorni[0],giorni[-1]))
# stampo anche a punti il dato raw
#ax.plot(trend['perc_guariti_su_positivi']['Italia']/trend['perc_deceduti_su_positivi']['Italia'],'bx')
#ax.legend(['Trend','Low','Up','Raw'])
ax.grid()


__main__:4: RuntimeWarning: invalid value encountered in true_divide


## Nuovi positivi *normalizzati*

In [20]:
from scipy.signal import savgol_filter
trend['perc_tamponi_positivi']['Italia'][0]=trend['perc_tamponi_positivi']['Italia'][1]

np_it=trend['nuovi_positivi']['Italia']


nwt=7
nwp=3

sm_np_it=moving_function_rev1(np_it,np.mean,nwt,nwt)

#gol_np_it=savgol_filter(np_it,13,2)

norm_np_it=0.01*np.multiply(moving_function_rev1(trend['tamponi_giornalieri']['Italia'],np.median,nwt,nwt),
                     moving_function_rev1(trend['perc_tamponi_positivi']['Italia'],np.mean,nwp,nwp))

norm_np_it=savgol_filter(norm_np_it,9,1)

C:\Users\klaat\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [21]:
plt.figure(figsize=(18,10))
plt.plot(np_it,'r.')
plt.plot(sm_np_it,'g--')
#plt.plot(gol_np_it,'g+-')
plt.plot(norm_np_it,'b')

plt.show()

In [22]:
trend['perc_tamponi_positivi']['Italia']

array([ 2.16329379,  2.16329379,  8.09128631, 10.30078286,  6.46563434,
        8.091706  , 22.95214923, 15.41929666, 18.55834329, 14.74503893,
       30.45544554, 19.46459845, 21.86568473, 18.94603175, 46.20725122,
       14.08795963, 18.6637618 , 20.61911799, 22.19221051, 29.93494265,
       22.8240829 , 24.74929189, 32.96867695, 24.91708126, 30.8772337 ,
       24.82890207, 24.89747874, 22.08101668, 28.06164303, 24.41849646,
       18.95855318, 16.80458828, 18.04718495, 16.85333032, 21.29040157,
       17.36036693, 13.68840555, 13.87897257, 11.72599161, 11.8730093 ,
       12.85618729, 12.60624471, 11.88926696,  9.01432682,  7.42260062,
        9.09090909,  7.38573698,  8.29196771,  8.75856164,  8.58730288,
       11.09824863,  6.1008807 ,  6.20665913,  5.31618598,  5.65573107,
        6.00891378,  5.43837235,  5.23539117,  5.34064436,  3.96951604,
        4.83770237,  3.60469206,  4.65582178,  5.43386558,  3.65099874,
        3.26820938,  2.73460325,  2.64796248,  3.42886017,  3.09